In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt  
from sklearn import model_selection
from sklearn import svm,ensemble, tree, neighbors
from sklearn import metrics
from sklearn.preprocessing import StandardScaler
from sklearn.naive_bayes import GaussianNB
from sklearn.metrics import accuracy_score
from sklearn.svm import LinearSVC
from sklearn.metrics import accuracy_score
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split 
from sklearn.model_selection import KFold

data_train = pd.read_csv('train.csv')
data_test = pd.read_csv('test.csv')

df = data_train.copy()
df_test = data_test.copy()


race_dic = {
    'Alpha': 'Mutant', #change in mutant
    'Amazon': 'Semihuman', #1 semi hmns
    'Asgardian': 'God', #New cat
    'Atlantean': 'Semihuman', #New cat
    'Bizarro': 'Semihuman', #1 semi hmns
    'Clone': 'Semihuman', #just one
    'Cosmic Entity': 'God', #no good
    'Cyborg': 'Android',
    'Dathomirian Zabrak': 'Alien', #alien? 1
    'Demi-God': 'God', 
    'Demon': 'Demon', #let it
    'Eternal': 'God', #God? 1
    'Flora Colossus': 'Alien', #alien? 1
    'God / Eternal': 'God',
    'Gorilla': 'Animal',
    'Gungan': 'Alien', #Alien? 1
    'Human / Altered': 'Semihuman', #1 semi hmns
    'Human / Clone': 'Semihuman', #1 semi hmns
    'Human / Cosmic': 'Semihuman', #1 semi hmns
    'Human / Radiation': 'Radioactive', #let it
    'Human-Kree': 'Semihuman', # semi hmns?
    'Human-Vuldarian': 'Semihuman', #1 semi hmns
    'Icthyo Sapien': 'Semihuman', #1 semi hmns
    'Inhuman' : 'Alien', #Alien?
    'Korugaran': 'Alien', #Alien?
    'Kryptonian': 'Alien', #Alien
    'Luphomoid': 'Semihuman', #1 semi hmns
    'Metahuman': 'Mutant', #mutant
    'Mutant': 'Mutant', #let it
    'New God': 'God',
    'Neyaphem': 'Demon', #Demon
    'Rodian': 'Alien', #alien
    'Spartoi': 'Alien', #alien
    'Symbiote': 'Alien', #alien
    'Talokite': 'Alien', #alien
    'Tamaranean': 'Alien', #alien
    'Ungaran': 'Alien', #alien
    'Vampire': 'Semihuman', #semihmn
    'Yautja': 'Alien', #alien
    "Yoda's species": 'Alien', #alien
    'Zen-Whoberian': 'Alien', #alien
    'Zombie': 'Semihuman' #Semihmns
    }

eye_color_dic = {
    'bown': "brown", 
    'gold': 'yellow',
    'hazel': 'green',
    'violet': 'purple',
    'yellow (without irises)': 'yellow',
    'silver' : 'grey', #
    'blue / white' : 'blue', #
    'amber' : 'brown', #
    'gold' : 'yellow', #
    'yellow / red' : 'red', #
    'yellow / blue' : 'yellow' #
    
     }


hair_color_dic = {
    'black': "Black",
    'Auburn': 'Red',
    'Magenta': 'Punk',
    'Silver': 'Grey',
    'Strawberry Blond': 'Blond',
    'blond': 'Blond',
    'Strawberry Blond' : 'Blond',
    'Gold' : 'Blond',
    'Red / Orange' : 'Punk',
    'Green' : 'Punk',
    'Purple' : 'Punk',
    'Red / Orange' : 'Red',
    'Blue' : 'Punk',
    'Brown / Black' : 'Brown',
    'Orange / White' : 'Punk',
    'Pink' : 'Punk',
    'Orange' : 'Red'
    }

skin_dic = {
    'pink': 'purple',
    'silver': 'grey',
    'red / black' : 'red',
    'orange / white' : 'red',
    'yellow' : 'gold',
    'orange' : 'red'
    }

publ_dic = {
    'SyFy': 'other',
    'Star Trek': 'other',
    'Wildstorm' : 'other',
    'Shueisha' : 'other',
    'ABC Studios' : 'other',
    'IDW Publishing' : 'other',
    'Team Epic TV' : 'other',
    'Rebellion' : 'other',
    'HarperCollins' : 'other',
    'Icon Comics' : 'other',
    'J. K. Rowling' : 'other'
    }



not_boolean_cols = ['Alignment', 'Gender', "Eye color", "Race", "Hair color",
                  "Publisher", "Skin color", "Height", "Weight", 'Id']

categorical_cols = ["Eye color", "Race", "Hair color",
                  "Publisher", "Skin color"]

quantitative_cols = ["Height", "Weight"]

def features_map(data):
    data.loc[:, 'Race'] = data['Race'].replace(race_dic)
    data.loc[:, 'Eye color'] = data['Eye color'].replace(eye_color_dic)
    data.loc[:, 'Hair color'] = data['Hair color'].replace(hair_color_dic)
    data['Skin Nan'] = data['Skin color'].isna()
    data.loc[:, 'Skin color'] = data['Skin color'].replace(skin_dic)
    data.loc[:, 'Publisher'] = data['Publisher'].replace(publ_dic)
    return data

def StringHash(a, m=257, C=1024):  # m=257, C=1024
# m represents the estimated cardinality of the items set
# C represents a number that is larger that ord(c)
    hash=0
    for i in range(len(a)):
        hash = (hash * C + ord(a[i])) % m
    return hash

def fillNaN_with_uniform(data, col):
    np.random.seed(42)
    a = data.values
    m = np.isnan(a) 
    mu, sigma = col.mean(), col.std()
    a[m] = np.random.normal(mu, sigma, size=m.sum())
    return data

def transform_categorical(data, column_name, method):
    if method == 'dummies':
        data[column_name] = data[column_name].fillna('NaN')
        f = pd.get_dummies(data[column_name])
        f = f.where(f['NaN'] == 0, np.nan)
        f = f.drop(columns='NaN')
        f = f.fillna(f.mean())
        return pd.concat([data, f],
                          axis=1).drop(columns=[column_name])
    elif method == 'hashing':
        data[column_name] = data[column_name].replace(np.nan, 'NaN')
        data[column_name] = data[column_name].apply(lambda x: StringHash(x))
        data[column_name] = data[column_name].replace(StringHash('NaN'), np.nan)
        data[column_name] = fillNaN_with_uniform(data[column_name], data[column_name])
        return data
    else:
        raise BaseException('Choose a method to handle categorical data.')

def important_boolean_cols(data, perc1, perc2):
    l = []
    d = data.drop(columns=not_boolean_cols)
    
    for i in d.columns:
        if d[i].sum() > perc1*len(d[i]):
            l.append(i)
        elif d[i].sum() > perc2*len(d[i]) and len(data[data[i] == True].Alignment.unique()) == 1:
            l.append(i)
    return l        
   
def test_processor(test):
    df_t = features_map(test)
    df_t.loc[df_t[df_t['Race'].str.contains(r'Human-.').fillna(False)].index, 'Race'] = 'Semihuman'
    df_t.loc[df_t[df_t['Race'].str.contains(r'Human /.').fillna(False)].index, 'Race'] = 'Semihuman'
    df_t.loc[df_t[df_t['Race'].str.contains(r'God /.').fillna(False)].index, 'Race'] = 'God'
    df_t.loc[df_t[df_t['Race'].str.contains(r'[^Human|Semihuman|Mutant|Android|God|Alien|Radioactive|Animal!Demon]').fillna(False)].index, 'Race'] = 'Alien'
    return df_t


def transform_dummies(data):
    features_map(data)
    data = transform_categorical(data, 'Race', 'dummies')
    data = transform_categorical(data, 'Eye color', 'dummies')
    data = transform_categorical(data, 'Hair color', 'dummies')
    data = transform_categorical(data, 'Skin color', 'dummies')
    data = transform_categorical(data, 'Publisher', 'dummies')
    return data

columns = quantitative_cols + ['Gender']
df = df[columns]
df.loc[:, 'Gender'] = df['Gender'].replace({'Female': 0, 'Male': 1})
df_test = df_test[columns]
df_test.loc[:, 'Gender'] = df_test['Gender'].replace({'Female': 0, 'Male': 1})
df['Train/Test'] = 1
df_test['Train/Test'] = 0
d = pd.concat([df, df_test]) * 1
d.loc[d["Height"] < 0, 'Height'] = np.nan
d.loc[d["Weight"] < 0, 'Weight'] = np.nan
H = d[(d["Height"] < 300) & (d["Weight"] < 600) & (d["Height"] > 100)].Height
W = d[(d["Height"] < 300) & (d["Weight"] < 600) & (d["Height"] > 100)].Weight
d["Height"] = fillNaN_with_uniform(d["Height"], H)
d["Weight"] = fillNaN_with_uniform(d["Weight"], W)
d["Gender"] = fillNaN_with_uniform(d["Gender"], d["Gender"])
d = d[d['Train/Test'] == 1].drop(columns='Train/Test')
data_train[columns] = d
data_train = data_train[not_boolean_cols + important_boolean_cols(data_train, 0.2, 0.02)]
#data_train = transform_dummies(data_train)
#data_train = data_train.drop(columns=categorical_cols)
data_train = data_train.drop(columns='Id')
features_map(data_train) #I call it just for hashing
transform_categorical(data_train, 'Race', 'hashing')
transform_categorical(data_train, 'Publisher', 'hashing')
transform_categorical(data_train, 'Skin color', 'hashing')
transform_categorical(data_train, 'Hair color', 'hashing')
transform_categorical(data_train, 'Eye color', 'hashing')
data_train = data_train*1
data_train

Alignment    Gender   Eye color        Race  Hair color  Publisher  \
0         bad  1.000000  210.125030   63.000000  202.952703      151.0   
1        good  1.000000  184.000000   74.000000  216.000000        3.0   
2        good  0.000000  229.000000   21.000000  112.000000        3.0   
3        good  0.000000   91.000000  163.000000  237.000000        3.0   
4        good  0.000000  164.000000  111.466708  196.000000        3.0   
5        good  0.000000   91.000000   84.519936  237.000000        3.0   
6         bad  1.000000  229.000000   74.000000  112.000000      223.0   
7        good  1.000000  229.000000  117.873654  112.000000        3.0   
8         bad  0.000000   91.000000  155.020777  237.000000        3.0   
9        good  0.000000  184.000000   80.450663  196.000000        3.0   
10    neutral  1.000000  184.000000  163.000000  171.000000        3.0   
11       good  0.000000  229.000000   74.000000  112.000000      223.0   
12       good  1.000000  229.000000   74.000000  112.000000      151.0   
13       good  1.000000   91.000000   80.451359  196.000000        3.0   
14       good  0.000000  184.000000   74.000000  216.000000        3.0   
15       good  0.000000  164.000000  157.405030  196.000000      223.0   
16       good  0.000000   91.000000   74.000000  237.000000      223.0   
17       good  1.000000  229.000000   74.000000  112.000000        3.0   
18       good  0.000000  169.933053  122.955359  168.575602        7.0   
19       good  0.000000  196.000000  125.000000  237.000000        3.0   
20       good  1.000000  229.000000   74.000000  112.000000        3.0   
21       good  1.000000  219.681195   70.464274  211.126306        3.0   
22       good  0.000000  229.000000  113.412284  237.000000        3.0   
23        bad  1.000000  275.087323   70.721304  258.516413      146.0   
24       good  1.000000  184.000000   70.623186  171.000000      223.0   
25       good  0.000000  163.863601  163.000000  163.384263        7.0   
26        bad  1.000000  196.000000  100.655726  196.000000        3.0   
27       good  0.000000  229.000000    9.193050  196.000000      223.0   
28       good  1.000000  184.000000   34.000000  216.000000        3.0   
29        bad  1.000000  184.000000  163.000000  112.000000        3.0   
..        ...       ...         ...         ...         ...        ...   
361       bad  1.000000  184.000000  124.910975  196.000000      223.0   
362      good  1.000000  276.037196   74.000000   34.230703        7.0   
363      good  1.000000  229.000000   38.152940  112.000000      223.0   
364      good  0.000000   91.000000   63.000000  237.000000      223.0   
365       bad  1.000000  229.000000   63.000000  216.000000        3.0   
366       bad  1.000000  235.000000  100.040286  216.000000      223.0   
367      good  1.000000  176.417047  145.859111  220.558076      223.0   
368      good  0.510878  184.000000   22.170250  196.000000        3.0   
369      good  1.000000  184.000000   74.000000  196.000000      223.0   
370   neutral  1.000000  229.000000  163.000000  112.000000        3.0   
371      good  1.000000  229.000000   74.000000  216.000000      151.0   
372      good  0.828837  277.721335   21.000000  180.773735        3.0   
373      good  0.000000  235.000000   98.222863  237.000000      223.0   
374      good  1.000000   12.863759  101.416224  159.873133       27.0   
375       bad  1.000000   91.000000  123.565953  171.000000        3.0   
376      good  1.000000  230.708346   37.894650  181.028930      223.0   
377      good  0.000000  184.000000   74.000000  216.000000      151.0   
378       bad  1.000000  164.000000   34.000000  171.000000      223.0   
379      good  1.000000  229.000000   34.350885  112.000000        3.0   
380      good  1.000000  235.000000  112.537291  171.000000        3.0   
381      good  1.000000  229.000000   74.000000  237.000000        3.0   
382      good  1.000000  229.000000   74.000000  112.000000        3.0   
383     

In [ ]:
from sklearn.datasets import make_classification
from sklearn.ensemble import ExtraTreesClassifier

X = data_train.drop(columns='Alignment')
y = data_train['Alignment']

#scaler = StandardScaler()
#X = scaler.fit_transform(X) 

# Build a forest and compute the feature importances
forest = ExtraTreesClassifier(n_estimators=250,
                              random_state=0)

forest.fit(X, y)
importances = forest.feature_importances_
std = np.std([tree.feature_importances_ for tree in forest.estimators_],
             axis=0)
indices = np.argsort(importances)[::-1]

# Print the feature ranking
print("Feature ranking:")

for f in range(X.shape[1]):
    print("%d. feature %d (%f)" % (f + 1, indices[f], importances[indices[f]]))

# Plot the feature importances of the forest
plt.figure()
plt.title("Feature importances")
plt.bar(range(X.shape[1]), importances[indices],
       color="r", yerr=std[indices], align="center")
plt.xticks(range(X.shape[1]), indices)
plt.xlim([-1, X.shape[1]])
plt.show()

In [6]:
X = data_train.drop(columns='Alignment')
y = data_train['Alignment']

X = X.iloc[:,[7,5,6,2,3,1,4,0,11,10]]
X_train, X_test, y_train, y_test = model_selection.train_test_split(X, y, train_size=0.7, random_state=42)
c1 = svm.SVC(C=10.0, gamma = 1e-5, random_state = 42)
c2 = svm.SVC(C=100.0, gamma = 1e-5,random_state=42)
c3 = svm.SVC(C=1000.0, gamma = 1e-6,random_state=42)
c4 = ensemble.GradientBoostingClassifier(random_state=42)
c5 = tree.DecisionTreeClassifier()
c6 = neighbors.KNeighborsClassifier(n_neighbors=2)
c7 = neighbors.KNeighborsClassifier(n_neighbors=7)
c8 = GaussianNB()
#c9 = LinearSVC(random_state=42)
c9 = RandomForestClassifier(n_estimators=100, random_state=42)
c10 = LogisticRegression()


scaler = StandardScaler()
X_train_scal = scaler.fit_transform(X_train) 
X_test_scal = scaler.fit_transform(X_test)
y_train = np.array(y_train)

methods = [c1, c2, c3, c4, c5, c6, c7, c8, c9, c10]
scores = []
acc_val = []
cv = KFold(n_splits=20, random_state=42, shuffle=False)

for i in methods:
    for train_index, test_index in cv.split(X_train_scal):
        X_train_cv, X_test_cv, y_train_cv, y_test_cv = X_train_scal[train_index], X_train_scal[test_index], y_train[train_index], y_train[test_index]
        i.fit(X_train_cv,y_train_cv)
        scores.append(i.score(X_test_cv, y_test_cv))

#sum(scores[40:60])/20 

for i in range(len(methods)):
    acc_val.append(sum(scores[(i*20):((i+1)*20)])/20)

#0.6516483516483517
#0.6115384615384614
#0.6516483516483517
print('The best method is {}, with validation accuracy {}'.format(methods[acc_val.index(max(acc_val))], max(acc_val)))    
acc_val

The best method is SVC(C=10.0, cache_size=200, class_weight=None, coef0=0.0,
    decision_function_shape='ovr', degree=3, gamma=1e-05, kernel='rbf',
    max_iter=-1, probability=False, random_state=42, shrinking=True, tol=0.001,
    verbose=False), with validation accuracy 0.6516483516483517


//anaconda3/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
//anaconda3/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:469: FutureWarning: Default multi_class will be changed to 'auto' in 0.22. Specify the multi_class option to silence this warning.
  "this warning.", FutureWarning)
//anaconda3/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
//anaconda3/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:469: FutureWarning: Default multi_class will be changed to 'auto' in 0.22. Specify the multi_class option to silence this warning.
  "this warning.", FutureWarning)
//anaconda3/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be ch

[0.6516483516483517,
 0.6516483516483517,
 0.6516483516483517,
 0.6085164835164836,
 0.5497252747252747,
 0.517032967032967,
 0.596978021978022,
 0.6104395604395603,
 0.6038461538461538,
 0.6478021978021978]

In [ ]:
c10.fit(X_train_scal, y_train)
c10.score(X_test_scal, y_test)

In [ ]:
X = data_train.drop(columns='Alignment')
y = data_train['Alignment']

X_train, X_test, y_train, y_test = model_selection.train_test_split(X, y, train_size=0.7, random_state=42)
X_train_val, X_val, y_train_val, y_val = model_selection.train_test_split(X_train, y_train, train_size=0.7, random_state=42)
print('The size of the training set is {}.'.format(X_train.shape))

scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train) 
X_train_val_scaled = scaler.fit_transform(X_train_val) 
X_test_scaled = scaler.fit_transform(X_test)
X_val_scaled = scaler.fit_transform(X_val)

clf1 = svm.SVC(C=10.0,gamma = 1e-5,random_state=42)
clf2 = svm.SVC(C=100.0,gamma = 1e-5,random_state=42)
clf3 = svm.SVC(C=1000.0,gamma = 1e-6,random_state=42)
clfGB = ensemble.GradientBoostingClassifier(random_state=42)
clft = tree.DecisionTreeClassifier()

clf1.fit(X_train_val_scaled,y_train_val)
yhat1 = clf1.predict(X_train_val_scaled)
acc1 = metrics.accuracy_score(y_train_val,yhat1)

clf2.fit(X_train_val_scaled,y_train_val)
yhat2 = clf2.predict(X_train_val_scaled)
acc2 = metrics.accuracy_score(y_train_val,yhat2)

clf3.fit(X_train_val_scaled,y_train_val)
yhat3 = clf3.predict(X_train_val_scaled)
acc3 = metrics.accuracy_score(y_train_val,yhat3)

clfGB.fit(X_train_val_scaled,y_train_val)
yhatGB = clfGB.predict(X_train_val_scaled)
accGB = metrics.accuracy_score(y_train_val,yhatGB)

clft.fit(X_train_val_scaled,y_train_val)
yhatt = clft.predict(X_train_val_scaled)
acct = metrics.accuracy_score(y_train_val,yhatt)
print('The train accuracy of 1 is {}, of 2 is {}, of 3 is {}. Of GB is {}. Of T is {}'.format(acc1, acc2, acc3, accGB, acct))

yhat1v = clf1.predict(X_val_scaled)
acc1v = metrics.accuracy_score(y_val,yhat1v)

yhat2v = clf2.predict(X_val_scaled)
acc2v = metrics.accuracy_score(y_val,yhat2v)

yhat3v = clf3.predict(X_val_scaled)
acc3v = metrics.accuracy_score(y_val,yhat3v)

yhatGBv = clfGB.predict(X_val_scaled)
accGBv = metrics.accuracy_score(y_val,yhatGBv)

yhattv = clft.predict(X_val_scaled)
acctv = metrics.accuracy_score(y_val,yhattv)
print('The validation accuracy of 1 is {}, of 2 is {}, of 3 is {}. Of GB is {}. Of T is {}'.format(acc1v, acc2v, acc3v, accGBv, acctv))

clf1.fit(X_train_scaled,y_train)
yhat1t = clf1.predict(X_test_scaled)
acc1t = metrics.accuracy_score(y_test,yhat1t)

clf2.fit(X_train_scaled,y_train)
yhat2t = clf2.predict(X_test_scaled)
acc2t = metrics.accuracy_score(y_test,yhat2t)

clf3.fit(X_train_scaled,y_train)
yhat3t = clf3.predict(X_test_scaled)
acc3t = metrics.accuracy_score(y_test,yhat3t)

clfGB.fit(X_train_scaled,y_train)
yhatGBt = clfGB.predict(X_test_scaled)
accGBt = metrics.accuracy_score(y_test,yhatGBt)

clft.fit(X_train_scaled,y_train)
yhattt = clft.predict(X_test_scaled)
acctt = metrics.accuracy_score(y_test,yhattt)
print(('The test accuracy of 1 is {}, of 2 is {}, of 3 is {}. Of GB is {}. Of T is {}.'.format(acc1t, acc2t, acc3t, accGBt, acctt)))
print('The best model has expected performance of', max([acc1t, acc2t, acc3t, accGBt, acctt]))

In [ ]:
r = pd.read_csv('train.csv')
r['Publisher'].value_counts()

In [ ]:

# CORE OF THE PROGRAM




''''
def prepare_data(data, test_size):
    y = data["Alignment"]
    X = data.drop(columns=['Alignment'])
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=test_size, random_state=42)
    X_train_scaled = StandardScaler().fit_transform(X_train)
    X_test_scaled = StandardScaler().fit_transform(X_test)
    return X_train_scaled, X_test_scaled, y_train, y_test

def compute_accuracy(method, X_train, X_test, y_train, y_test):
    method.fit(X_train, y_train) 
    yhat = method.predict(X_test)
    return metrics.accuracy_score(y_test,yhat)

def run(**kwargs):
    data = pd.read_csv('train.csv')[chosen_columns]
    test_data = pd.read_csv('test.csv')[chosen_columns[1:]] #Dropping 'Alignment' column.
    cleaned_data = clean_data(data, **kwargs)
    cleaned_test_data = clean_data(test_data, **kwargs)
    X_train, X_test, y_train, y_test = prepare_data(cleaned_data, kwargs['test_size'])
    max_acc = 0.0
    for name, method in kwargs['methods'].items():
        acc = compute_accuracy(method, X_train, X_test, y_train, y_test)
        print('The accuracy for {} is {}.'.format(name, acc))
        if acc > max_acc:
            max_acc = acc
            best_model = name
    print('')
    print('RESULT:')
    print('The {} is the model with best accuracy, acc = {}.'.format(best_model, max_acc))

# METHODS

methods = {
    'SVC with C = 10': svm.SVC(C = 10.0, gamma = 1e-5, random_state = 42),
    'SVC with C = 100': svm.SVC(C=100.0,gamma = 1e-5,random_state=42),
    'SVC with C = 1000': svm.SVC(C=1000.0,gamma = 1e-6,random_state=42),
    'GradientBoosting': ensemble.GradientBoostingClassifier(random_state=42),
    'DecisionTree': tree.DecisionTreeClassifier(),
    'KNeighbors with n = 2': neighbors.KNeighborsClassifier(n_neighbors=2),
    'NNeighbors with n = 7': neighbors.KNeighborsClassifier(n_neighbors=7),
    'Gaussian Naive-Bayes': GaussianNB(),
    'LinearSVC': LinearSVC(random_state=42),
    'RandomForest': RandomForestClassifier(n_estimators=100, random_state=3),
    'LogisticRegression': LogisticRegression(),
}

# HUMAN MANIPULATIONS - TECHNICAL INFO

chosen_columns = ["Alignment", "Gender", "Eye color", "Race", "Hair color",
                  "Publisher", "Skin color", "Height", "Weight", "Agility",
                  "Accelerated Healing", "Lantern Power Ring",
                  "Dimensional Awareness", "Cold Resistance", "Durability",
                  "Stealth", "Energy Absorption", "Flight", "Danger Sense"]

eye_color_dic = {
    'bown': "brown", 
    'gold': 'yellow',
    'hazel': 'green',
    'violet': 'purple',
    'yellow (without irises)': 'yellow',
}

hair_color_dic = {
    'black': "Black",
    'Auburn': 'Red',
    'Magenta': 'Purple',
    'Pink': 'Purple',
    'Silver': 'Grey',
    'Strawberry Blond': 'Blond',
    'blond': 'Blond'
}

skin_dic = {
    'pink': 'purple',
    'silver': 'grey'
}

race_dic = {
    'Alpha': 'No idea',
    'Amazon': 'Human',
    'Asgardian': 'Human',
    'Atlantean': 'Human',
    'Bizarro': 'No idea',
    'Clone': 'Android',
    'Cosmic Entity': 'God',
    'Cyborg': 'Android',
    'Dathomirian Zabrak': 'No idea',
    'Demi-God': 'God',
    'Demon': 'Bad',
    'Eternal': 'God',
    'Flora Colossus': 'God',
    'God / Eternal': 'God',
    'Gorilla': 'Animal',
    'Gungan': 'No idea',
    'Human / Altered': 'Human',
    'Human / Clone': 'Human',
    'Human / Cosmic': 'Human',
    'Human / Radiation': 'Human',
    'Human-Kree': 'Human-Kree',
    'Human-Vuldarian': 'Human',
    'Icthyo Sapien': 'No idea',
    'Inhuman' : 'Bad',
    'Korugaran': 'No idea',
    'Kryptonian': 'Human',
    'Luphomoid': 'No idea',
    'Metahuman': 'Human',
    'Mutant': 'Human',
    'New God': 'God',
    'Neyaphem': 'No idea',
    'Rodian': 'Human',
    'Spartoi': 'No idea',
    'Symbiote': 'No idea',
    'Talokite': 'No idea',
    'Tamaranean': 'No idea',
    'Ungaran': 'No idea',
    'Vampire': 'Bad',
    'Yautja': 'No idea',
    "Yoda's species": 'No idea',
    'Zen-Whoberian': 'No idea',
    'Zombie': 'Bad'
}

# FUNCTIONS TO CLEAN DATA

def transform_categorical(data, column_name, method):
    if method == 'dummies':
        return pd.concat([data, pd.get_dummies(data[column_name])],
                          axis=1).drop(columns=[column_name])
    elif method == 'hashing':
        raise BaseException('Needs implementation.') # Pablo is going to implement this.
    else:
        raise BaseException('Choose a method to handle categorical data.')

def generate_distribution(data):
    np.random.seed(42)
    return np.random.normal(data.mean(), data.std()) # Not sure normal is the best dist., we should check.

def clean_data(data, **kwargs):
    data = data * 1 # With this command we transform all True False values to 0 or 1
    data.loc[:, 'Gender'] = data['Gender'].map({'Female': 0, 'Male': 1})
    data['Gender'].fillna(generate_distribution(data['Gender']), inplace=True)
    data.loc[data["Height"] < 0, 'Height'] = np.nan
    data["Height"].fillna(generate_distribution(data["Height"]), inplace=True)
    data.loc[data["Weight"] < 0, 'Weight'] = np.nan
    data["Weight"].fillna(generate_distribution(data['Weight']), inplace=True)
    data.loc[:, 'Eye color'] = data["Eye color"].map(kwargs['eyes_map'])
    data.loc[:, 'Eye color'] = data.loc[:, 'Eye color'] + '_eye'
    data = transform_categorical(data, 'Eye color', kwargs['categorical_handler'])
    data.loc[:, 'Hair color'] = data["Hair color"].map(kwargs['hairs_map'])
    data.loc[:, 'Hair color'] = data.loc[:, 'Hair color'] + '_hair'
    data = transform_categorical(data, 'Hair color', kwargs['categorical_handler'])
    data.loc[:, 'Race'] = data['Race'].map(kwargs['races_map'])
    data = transform_categorical(data, 'Race', kwargs['categorical_handler'])
    data = transform_categorical(data, 'Publisher', kwargs['categorical_handler'])
    data.loc[:, 'Skin color'] = data['Skin color'].map(kwargs['skins_map'])
    data.loc[:, 'Skin color'] = data.loc[:, 'Skin color'] + '_skin'
    data = transform_categorical(data, 'Skin color', kwargs['categorical_handler'])
    return data

## Parameters

parameters = {
    'methods': methods,
    'columns': chosen_columns,
    'test_size': 0.2,
    'categorical_handler': 'dummies',
    'eyes_map': eye_color_dic,
    'hairs_map': hair_color_dic,
    'races_map': race_dic,
    'skins_map': skin_dic,
}

# RUNNER

run(**parameters)


clean_data(data)